### Downloads Files
This file contains code to download all the remaining files (HMDA and Coastal downloaded in HMDA_Download.ipynb).<br/>
Saves files in organized folders so that they can be used in further analysis.<br/>

#### Required variables can be set before invoking with %run:
<strong>SLRs</strong><br>
List of all file extensions for SLR files to download - (Dynamic JavaScript so didn't scrape)</strong><br>
<strong>fips_dict</strong><br>
Dictionary of State-FIPS codes for all states to download<br>

In [1]:
#imports
import zipfile
from bs4 import BeautifulSoup
import requests

In [6]:
#Setting Default parameters if no parameters specified when called to run
try:
    SLRs
except:  
    SLRs = ['AL/AL','CA/CA_EKA','CA/CA_LOX','CA/CA_MTR','CA/CA_SGX','CA/CA_ChannelIslands','CA/CA_Catalina',
        'NewEngland/CT','DE/DE','DC/DC','FL/FL_Pan_West','FL/FL_Pan_East','FL/FL_West_3','FL/FL_West_2',
        'FL/FL_West_1','FL/FL_SW','FL/FL_Keys','FL/FL_SE','FL/FL_East_1','FL/FL_East_2','FL/FL_NE',
        'GA/GA_South','GA/GA_North','LA/LA_LP','LA/LA_Delta','LA/LA_Central','LA/LA_CentralEast',
        'LA/LA_CentralNorth','LA/LA_West','NewEngland/ME_East','NewEngland/ME_West','MD/MD_Southeast',
        'MD/MD_East','MD/MD_North','MD/MD_West','MD/MD_Southwest','NewEngland/MA','MS/MS','NewEngland/NH',
        'NJ/NJ_Northern','NJ/NJ_Middle','NJ/NJ_Southern','NY/NY_Hudson','NY/NY_Metro','NY/NY_Suffolk',
        'NC/NC_Northern','NC/NC_Middle1','NC/NC_Middle2','NC/NC_Southern1','NC/NC_Southern2','OR/OR_MFR',
        'OR/OR_PQR','PA/PA','NewEngland/RI','SC/SC_North','SC/SC_Central','SC/SC_South','TX/TX_North1',
        'TX/TX_North2','TX/TX_Central','TX/TX_South1','TX/TX_South2','VA/VA_EasternShore','VA/VA_Northern',
        'VA/VA_Middle','VA/VA_Southern','WA/WA_PQR','WA/WA_SEW','WA/WA_PugetNW','WA/WA_PugetSW']

try:
    fips_dict
except: 
    fips_dict = {'AL':'01','CA':'06','CT':'09','DE':'10','DC':'11','FL':'12','GA':'13','LA':'22','ME':'23',
             'MD':'24','MA':'25','MS':'28','NH':'33','NJ':'34','NY':'36','NC':'37','OR':'41','PA':'42','RI':'44',
             'SC':'45','TX':'48','VA':'51','WA':'53'}

In [9]:
#Should already be created, but better safe than sorry
def data_directory():
    ! mkdir data

In [11]:
data_directory()

mkdir: cannot create directory ‘data’: File exists


In [16]:
def download_FEMA():
    
    ! mkdir data/FEMA
    url = 'https://hazards.fema.gov/nri/Content/StaticDocuments/DataDownload//NRI_Table_CensusTracts/NRI_Table_CensusTracts.zip'
    r = requests.get(url, allow_redirects=True)
    with open('data/FEMA/NRI_Table_CensusTracts.zip', 'wb') as file:
        file.write(r.content)
        archive = zipfile.ZipFile('data/FEMA/NRI_Table_CensusTracts.zip')
        for file in archive.namelist():
            if file.endswith('CensusTracts.csv'):
                archive.extract(file, 'data/FEMA')

In [17]:
download_FEMA()

In [19]:
def download_SLR():

    ! mkdir data/SLR_ShapeFiles

    for stem in SLRs:
        slr = stem.split('/')[1]
        state = slr[:2]
        zip_link = 'https://chs.coast.noaa.gov/htdata/Inundation/SLR/SLRdata/{}_slr_data_dist.zip'.format(stem)
        r = requests.get(zip_link, allow_redirects=True)
        with open('data/SLR_ShapeFiles/{}_slr_data_dist.zip'.format(slr), 'wb') as file:
            file.write(r.content)
            archive = zipfile.ZipFile('data/SLR_ShapeFiles/{}_slr_data_dist.zip'.format(slr))
            for file in archive.namelist():
                if file.endswith('.gdbtable') or file.endswith('.gpkg'):
                    archive.extract(file, 'data/SLR_ShapeFiles')         

In [20]:
download_SLR(SLRs)

In [21]:
def download_Census():

    ! mkdir data/Census_ShapeFiles

    parser = 'html.parser'
    URL = "https://www2.census.gov/geo/tiger/TIGER_RD18/LAYER/TRACT/"
    r = requests.get(URL)
 
    soup = BeautifulSoup(r.content, parser)
    for link in soup.find_all('a', href=True):
        tract = link['href']
        if tract.endswith('.zip'):
            fip = tract.split('_')[2]
            #only download shape files for speficied states
            if fip in fips_dict.values():
                zip_file = 'https://www2.census.gov/geo/tiger/TIGER_RD18/LAYER/TRACT/{}'.format(tract)
                r = requests.get(zip_file, allow_redirects=True)
                with open('data/Census_ShapeFiles/{}'.format(tract), 'wb') as file:
                    file.write(r.content)
                    archive = zipfile.ZipFile('data/Census_ShapeFiles/{}'.format(tract))
                    for file in archive.namelist():
                        archive.extract(file, 'data/Census_ShapeFiles')

In [22]:
download_Census(fips_dict)